In [1]:
from deap import gp, creator, tools, base, algorithms
from random import randint, random, seed, getstate, setstate
from functools import partial
from itertools import repeat

pset = gp.PrimitiveSetTyped(name='MAIN',
                            in_types=repeat(int, 1),
                            ret_type=float)
pset.addPrimitive(name='add', primitive=None, ret_type=float, in_types=[float, float])
pset.addPrimitive(name='sub', primitive=None, ret_type=float, in_types=[float, float])
pset.addPrimitive(name='mul', primitive=None, ret_type=float, in_types=[float, float])
pset.addPrimitive(name='div', primitive=None, ret_type=float, in_types=[float, float])
pset.addPrimitive(name='pow', primitive=None, ret_type=float, in_types=[float, float])
pset.addPrimitive(name='sqrt', primitive=None, ret_type=float, in_types=[float])
pset.addPrimitive(name='and', primitive=None, ret_type=bool, in_types=[bool, bool])
pset.addPrimitive(name='or', primitive=None, ret_type=bool, in_types=[bool, bool])
pset.addPrimitive(name='not', primitive=None, ret_type=bool, in_types=[bool])
pset.addPrimitive(name='tern', primitive=None, ret_type=float, in_types=[bool, float, float])
pset.addPrimitive(name='lt', primitive=None, ret_type=bool, in_types=[float, float])
pset.addPrimitive(name='gt', primitive=None, ret_type=bool, in_types=[float, float])
pset.addPrimitive(name='eq', primitive=None, ret_type=bool, in_types=[float, float])

pset.addEphemeralConstant(name='iconst', ret_type=float, ephemeral=partial(randint, 0, 10))
pset.addEphemeralConstant(name='rconst', ret_type=float, ephemeral=partial(random))
pset.addEphemeralConstant(name='bconst', ret_type=bool, ephemeral=partial(randint, 0, 1))

pset.addTerminal(name='original', terminal=None, ret_type=float)
pset.addTerminal(name='cost', terminal=None, ret_type=float)
pset.addTerminal(name='degree', terminal=None, ret_type=float)
pset.addTerminal(name='area', terminal=None, ret_type=float)

In [2]:
creator.create('FitnessMin', base.Fitness, weights=(-1.0,))
creator.create('Individual', gp.PrimitiveTree, fitness=creator.FitnessMin)

In [3]:
import numpy as np
import subprocess as sp
import scipy.stats as st
import multiprocessing as mp

CAMINHO = r'/home/mpvreal/Code/Faculdade/tcc/llvm/testes/x86-64'
ALOCADOR = 'greedy'

individuos_compilados = dict[str, str]()

def obter_scripts_benchmark(benchmark: str) -> dict[str, str]:
  """
  Obtém os scripts de compilação e execução do benchmark.
  """
  global CAMINHO
  
  return { 'compile': f'{CAMINHO}/{benchmark}/compile', 'run': f'{CAMINHO}/{benchmark}/runbench' }


def compilar_individuo(benchmark: str, individuo) -> None:
  """
  Escreve função heurística da alocação em um arquivo e compila o benchmark.
  """
  global ALOCADOR
  global individuos_compilados
  heuristica = str(gp.PrimitiveTree(individuo))
  script = obter_scripts_benchmark(benchmark)['compile']

  if heuristica not in individuos_compilados:
    executavel = sp.run(f'{script} {ALOCADOR} "{heuristica}"', shell=True,
                        stdout=sp.PIPE).stdout.decode('utf-8').strip()
    individuos_compilados[heuristica] = executavel


def calcular_media_e_erro(dados, confianca=0.95):
  """
  Obtém a média e margem de erro de um conjunto de dados, com um intervalo de confiança de 95%.
  """
  a = 1.0 * np.array(dados)
  n = len(a)

  media, erro_medio = np.mean(a), st.sem(a)
  h = erro_medio * st.t.ppf((1 + confianca) / 2., n-1)

  return media, h


def avaliar_fitness(benchmark: str, individuo, verbose):
  """
  Escreve a heurística em um arquivo, compila o benchmark e coleta o tempo de execução médio após
  30 rodadas.
  """
  global individuos_compilados

  heuristica = str(gp.PrimitiveTree(individuo))
  try:
    executavel = individuos_compilados[heuristica]
  except KeyError:
    return float('inf'),
  scripts = obter_scripts_benchmark(benchmark)
  tempos = []
  for i in range(30):
    if verbose:
      print(f'Avaliando {heuristica}, rodada {i + 1}')
    tempos.append(float(sp.run([scripts['run'], executavel], 
                  stdout=sp.PIPE).stdout.decode('utf-8').replace(',', '.').strip()))

  media, margem_erro = calcular_media_e_erro(tempos)

  return media,
from operator import attrgetter

def inicializar_individuo(classe, string, pset):
  """
  Inicializa um indivíduo com uma árvore de expressão a partir de uma string.
  """
  return classe(gp.PrimitiveTree.from_string(string, pset))

def inicializar_populacao(inicializar, populacao, pset):
  """
  Inicializa a população com árvores de expressão a partir do arquivo populacao.
  """
  with open(populacao, 'r') as f:
    pop = f.read().split('\n')
  gp.PrimitiveTree
  
  return [inicializar(i, pset) for i in pop]

tb = base.Toolbox()
pool = mp.Pool()
tb.register('map', pool.map)
tb.register('individual', inicializar_individuo, creator.Individual)
tb.register('population', inicializar_populacao, tb.individual, '../populacao.txt', pset)
tb.register('compile', compilar_individuo, '519.lbm_r')
tb.register('evaluate', avaliar_fitness, '519.lbm_r', verbose=True)
tb.register('select', tools.selTournament, tournsize=7)
tb.register('mate', gp.cxOnePoint)
tb.register('expr_mut', gp.genFull, min_=0, max_=2)
tb.register('mutate', gp.mutUniform, expr=tb.expr_mut, pset=pset)
tb.decorate('mate', gp.staticLimit(key=attrgetter('height'), max_value=17))
tb.decorate('mutate', gp.staticLimit(key=attrgetter('height'), max_value=17))

In [4]:
import pickle

CHECKPOINT_FREQ = 5 # Salvar a população a cada x gerações

def salvar_checkpoint(checkpoint, pop, gen, hof, log):
  global CHECKPOINT_FREQ
  if checkpoint and gen % CHECKPOINT_FREQ == 0:
    with open(checkpoint, 'wb') as arquivo_cp:
      cp = dict(population=pop, 
                generation=gen, 
                halloffame=hof,
                logbook=log,
                rndstate=getstate())
      pickle.dump(cp, arquivo_cp)

def evoluir(population, toolbox, mu, lambda_, cxpb, mutpb, ngen,
            stats=None, halloffame=None, verbose=__debug__, checkpoint=None, logbook=None):
  logbook_from_checkpoint = logbook is not None
  
  if not logbook_from_checkpoint:
    logbook = tools.Logbook()
    logbook.header = ['gen', 'nevals'] + (stats.fields if stats else [])

  # Evaluate the individuals with an invalid fitness
  invalid_ind = [ind for ind in population if not ind.fitness.valid]
  for ind in invalid_ind:
    toolbox.compile(ind)
  fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
  for ind, fit in zip(invalid_ind, fitnesses):
    ind.fitness.values = fit

  if halloffame is not None:
    halloffame.update(population)

  record = stats.compile(population) if stats is not None else {}
  if not logbook_from_checkpoint:  
    logbook.record(gen=0, nevals=len(invalid_ind), **record)
    if verbose:
      print(logbook.stream)

  # Begin the generational process
  for gen in range(1, ngen + 1):
    # Vary the population
    offspring = algorithms.varOr(population, toolbox, lambda_, cxpb, mutpb)

    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    for ind in invalid_ind:
      toolbox.compile(ind)
    fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
      ind.fitness.values = fit

    # Update the hall of fame with the generated individuals
    if halloffame is not None:
      halloffame.update(offspring)

    # Select the next generation population
    population[:] = toolbox.select(population + offspring, mu)

    # Update the statistics with the new population
    record = stats.compile(population) if stats is not None else {}
    logbook.record(gen=gen, nevals=len(invalid_ind), **record)
    if verbose:
      print(logbook.stream)

    # Salvar a população a cada x gerações
    salvar_checkpoint(checkpoint, population, gen, halloffame, logbook)

  return population, logbook

seed(666)

if True:
  pop = tb.population()
  hof = tools.HallOfFame(1)
  start_gen = 0
  logbook = None

  stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
  stats_size = tools.Statistics(len)
  mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
  mstats.register('media', np.mean)
  mstats.register('dvp', np.std)
  mstats.register('min', np.min)
  mstats.register('max', np.max)
else:
  with open(checkpoint, "rb") as cp_file:
    cp = pickle.load(cp_file)
  pop = cp["population"]
  start_gen = cp["generation"]
  hof = cp["halloffame"]
  logbook = cp["logbook"]
  setstate(cp["rndstate"])

pop, log = evoluir(population=pop,
                   toolbox=tb,
                   cxpb=0.5,
                   mutpb=0.1,
                   mu=len(pop),
                   lambda_=(2 * len(pop)),
                   ngen=30 - start_gen,
                   stats=mstats,
                   halloffame=hof,
                   verbose=True,
                   logbook=logbook,
                   checkpoint='checkpoint.pkl')

function@main,0,0,0,0,0,14,0,0,0,0,54.1667
function@LBM_allocateGrid,0,0,0,0,0,1,0,0,0,0,1
function@LBM_freeGrid,0,0,0,0,0,1,0,0,0,0,1
function@LBM_initializeGrid,0,0,0,0,0,0,0,0,0,0,0
function@MAIN_parseCommandLine,0,0,0,0,0,3,0,0,0function@LBM_swapGrids,0,0,0,0,0,3,0,0,0,0,0,0,0

function@MAIN_printInfo,0,0,0,0,0,0,function@LBM_loadObstacleFile,00,,00,,00,,00,,00,
9,0,0,0,0,33860.5
function@MAIN_initialize,0,0,0,0,0,1,0,0,0,0,1
function@MAIN_finalize,0,0,0,0,0,1,0,0,0,0,1
function@LBM_initializeSpecialCellsForLDC,0,0,0,0,0,7,0,0,0,0,3740.12
function@LBM_initializeSpecialCellsForChannel,0,0,0,0,0,2,0,0,0,0,10954.5
function@LBM_performStreamCollideBGK,4,4,0,6,0,3,31.5,31.5,95.25,0,47.625
function@LBM_performStreamCollideTRT,28,27,0,49,0,11,444.5,420.625,777.875,0,158.625
function@LBM_handleInOutFlow,6,6,0,11,0,4,191.25,191.25,350.625,0,127.5
function@LBM_showGridStatistics,21,3,0,15,0,6,148.75,47.8125,252.938,0,19.9375
function@LBM_storeVelocityField,3,0,0,3,0,12,1088,0,1088,0,114694
f

   	      	                   fitness                    	                     size                     
   	      	----------------------------------------------	----------------------------------------------
gen	nevals	dvp	gen	max	media	min	nevals	dvp	gen	max	media	min	nevals
0  	6     	nan	0  	inf	inf  	inf	6     	0  	0  	5  	5    	5  	6     


function@LBM_handleInOutFlow,4,8,0,11,0,5,127.5,255,350.625,0,159.375
function@LBM_showGridStatistics,20,3,0,15,0,6,148.75,47.8125,252.938,0,19.9375
function@LBM_storeVelocityField,3,0,0,3,0,12,1088,0,1088,0,114694
function@LBM_compareVelocityField,22,1,0,23,0,14,623680,32768,656448,0,139270
function@LBM_allocateGrid,0,0,0,0,0,1,0,0,0,0,1
function@LBM_freeGrid,0,0,0,0,0,1,0,0,0,0,1function@main,0,0,0,0,0,14,0
,0,0,0,54.1667
function@LBM_initializeGrid,0,0,0,0,0,0,0,0,0,0,0
function@LBM_swapGrids,0,0,0,0,0,0,0,0,0,0,0
function@MAIN_parseCommandLine,0,0,0,function@LBM_loadObstacleFile,0,0,0,0,0,9,0,0,0,0,33860.5
0,0,3,0,0,0,0,3
function@MAIN_printInfo,0,0,0,0,0,0,0,0,0,0,0
function@LBM_initializeSpecialCellsForLDC,0,0,0,0,0,7,0,0,0,0,3740.12
function@MAIN_initialize,0,0,0,0,0,1,0,0,0,0,1
function@MAIN_finalize,0,0,0,0,0,1,0,0,0,0,1
function@LBM_initializeSpecialCellsForChannel,0,0,0,0,0,2,0,0,0,0,10954.5
function@LBM_performStreamCollideBGK,3,9,0,8,0,9,23.625,70.875,127,0,94.875
function

1  	9     	nan	1  	inf	inf  	inf	9     	0  	1  	5  	5    	5  	9     


function@main,0,function@LBM_allocateGrid,0,0,0,0,0,0,14,0,0,0,0,54.1667
0,0,0,1,0,0,0,0,1
function@LBM_freeGrid,0,0,0,0,0,1,0,0,0,0,1
function@MAIN_parseCommandLine,0,0,0,0,0,3,0,0,0,0,3
function@LBM_initializeGrid,0,0,0,0,0,0,0,0,0,0,0
function@LBM_swapGrids,0,0,0,0,0,0,0,0,0,0,0
function@MAIN_printInfo,0,0,0,0,0,0,0,0,0,0,0
function@function@MAIN_initialize,0,0,0,0,0,1,0,0,0,0,1
LBM_loadObstacleFile,0,0,0,0,0,9,0,0,0,0,33860.5
function@MAIN_finalize,0,0,0,0,0,1,0,0,0,0,1
function@LBM_initializeSpecialCellsForLDC,0,0,0,0,0,7,0,0,0,0,3740.12
function@LBM_initializeSpecialCellsForChannel,0,0,0,0,0,2,0,0,0,0,10954.5
function@LBM_performStreamCollideBGK,2,5,0,6,0,9,15.75,39.375,95.25,0,94.875
function@LBM_performStreamCollideTRT,29,31,0,51,0,4,460.375,468.125,809.625,0,63.5
function@LBM_handleInOutFlow,3,9,0,9,0,4,95.625,286.875,286.875,0,127.5
function@LBM_showGridStatistics,21,3,0,15,0,6,148.75,47.8125,252.938,0,19.9375
function@LBM_storeVelocityField,3,0,0,5,0,14,1088,0,1121,0,115750


...

21 	11    	nan	21 	inf	inf  	inf	11    	0.763763	21 	7  	6.5    	5  	11    


function@LBM_allocateGrid,0,0,0,0,0,1,0,0,0,0,1
function@main,0,0,0,0,0,14,0,0,0,0,54.1667
function@LBM_freeGrid,0,0,0,0,0,1,0,0,0,0,1
function@LBM_initializeGrid,0,0,0,0,0,0,0,0,0,0,0
function@LBM_swapGrids,0,0,0,0,0,0,0,0,0,0,0
function@MAIN_parseCommandLine,0,0,0,0,0,3,0,0,0,0,3
function@LBM_loadObstacleFile,0,0,0,0,0,9,0,0,0,0,33860.5
function@MAIN_printInfo,0,0,0,0,0,0,0,0,0,0,0
function@MAIN_initialize,0,0,0,0,0,1,0,0,0,0,1
function@MAIN_finalize,0,0,0,0,0,1,0,0,0,0,1
function@LBM_initializeSpecialCellsForLDC,0,0,0,0,0,7,0,0,0,0,3740.12
function@LBM_initializeSpecialCellsForChannel,0,0,0,0,0,2,0,0,0,0,10954.5
function@LBM_performStreamCollideBGK,2,4,0,5,0,3,15.75,31.5,79.375,0,47.625
function@LBM_performStreamCollideTRT,28,27,0,49,0,4,444.5,412.625,777.875,0,79.5
function@LBM_handleInOutFlow,4,5,0,9,0,8,127.5,159.375,286.875,0,255
function@LBM_showGridStatistics,21,2,0,14,0,6,148.75,31.875,221,0,19.9375
function@LBM_storeVelocityField,3,0,0,3,0,13,1088,0,1088,0,114726
function@LB

22 	6     	nan	22 	inf	inf  	inf	6     	1.86339 	22 	11 	7.16667	5  	6     


PLEASE submit a bug report to https://github.com/llvm/llvm-project/issues/ and include the crash backtrace.
Stack dump:
0.	Program arguments: /home/mpvreal/Code/Faculdade/tcc/llvm/build/bin/llc -O3 -filetype=obj "--priority-function=div(tern(0, pow(pow(degree, 2.0), 2.0), original), original)" --regalloc=greedy ./main.o.ll
1.	Running pass 'Function Pass Manager' on module './main.o.ll'.
2.	Running pass 'Greedy Register Allocator' on function '@main'
PLEASE submit a bug report to https://github.com/llvm/llvm-project/issues/ and include the crash backtrace.
Stack dump:
0.	Program arguments: /home/mpvreal/Code/Faculdade/tcc/llvm/build/bin/llc -O3 -filetype=obj "--priority-function=div(tern(0, pow(pow(degree, 2.0), 2.0), original), original)" --regalloc=greedy ./lbm.o.ll
1.	Running pass 'Function Pass Manager' on module './lbm.o.ll'.
2.	Running pass 'Greedy Register Allocator' on function '@LBM_allocateGrid'
 #0 0x000055ad9863fef6 llvm::sys::PrintStackTrace(llvm::raw_ostream&, int) /home/m

KeyboardInterrupt: 